In [1]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [2]:
num_nodes = 4

num_txns = 7200
fialed_node_id = 3
write_percent = 10
usecase = 'stack'
operationrate =1000000000

In [3]:
gcp_username = 'tejas'

In [6]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create tmane-'+f"{i:03}"+' \
    --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-a \
    --machine-type=e2-highcpu-4 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20241002-130909,image=projects/rhel-cloud/global/images/rhel-8-v20240910,mode=rw,size=20,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# # Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
# with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
#     # Submit each command to the executor for parallel execution
#     futures = [executor.submit(run_command, command) for command in commands]

#     # Wait for all tasks to complete
#     concurrent.futures.wait(futures)

In [ ]:
# time.sleep(60)

In [5]:
# os.system('git add .; git commit -m "test"; git push -f origin HEAD:main')


[main c225bce] test
 1 file changed, 29 insertions(+), 2612 deletions(-)


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
   1a5cf57..c225bce  HEAD -> main


0

In [ ]:
def git_clone(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "sudo yum update; sudo yum -y install git; git clone --branch main --single-branch https://github.com/tejas-shivanand-mane/Optimistic-Replication.git;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_clone)(i) for i in range(num_nodes))


In [7]:
def kill_code(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "sudo killall main-replication-protocols"'
    )
    output = subprocess.getoutput(command)
    print(output)


## RESTART

In [42]:

for i in range(num_nodes):
    kill_code(i)

    
os.system('git add .; git commit -m "older main, old scl"; git push -f origin HEAD:main')

def git_pull(i):

    command = 'gcloud compute ssh --zone "us-central1-a" "tmane-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"'

    print(command)

    output = os.system(command)
    print(output)
results = Parallel(n_jobs=60, prefer="threads")(delayed(git_pull)(i) for i in range(num_nodes))

main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
[main eb63e20] older main, old scl
 2 files changed, 107 insertions(+), 134 deletions(-)


To github.com:tejas-shivanand-mane/Optimistic-Replication.git
   67a2de3..eb63e20  HEAD -> main


gcloud compute ssh --zone "us-central1-a" "tmane-003" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-001" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-000" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"
gcloud compute ssh --zone "us-central1-a" "tmane-002" --project "ucr-ursa-major-lesani-lab" --command "cd Optimistic-Replication ; git pull;"


From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   67a2de3..eb63e20  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   67a2de3..eb63e20  main       -> origin/main


Updating 67a2de3..eb63e20
Fast-forward
 OptimisticReplication.ipynb        | 225 +++++++++++++++----------------------
 tcp/main-replication-protocols.cpp |  16 +++
 2 files changed, 107 insertions(+), 134 deletions(-)
0
Updating 67a2de3..eb63e20
Fast-forward
 OptimisticReplication.ipynb        | 225 +++++++++++++++----------------------
 tcp/main-replication-protocols.cpp |  16 +++
 2 files changed, 107 insertions(+), 134 deletions(-)
0
Updating 67a2de3..eb63e20
Fast-forward
 OptimisticReplication.ipynb        | 225 +++++++++++++++----------------------
 tcp/main-replication-protocols.cpp |  16 +++
 2 files changed, 107 insertions(+), 134 deletions(-)
Updating 67a2de3..eb63e20
Fast-forward
 OptimisticReplication.ipynb        | 225 +++++++++++++++----------------------
 tcp/main-replication-protocols.cpp |  16 +++
 2 files changed, 107 insertions(+), 134 deletions(-)
0


From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   67a2de3..eb63e20  main       -> origin/main
From https://github.com/tejas-shivanand-mane/Optimistic-Replication
   67a2de3..eb63e20  main       -> origin/main


0


In [43]:
def setup(i):
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication; cp optrep_script.sh ../optrep_script.sh; cd ../; sh optrep_script.sh {num_nodes} {num_txns} {write_percent} {usecase} > /home/{gcp_username}/log.txt"'
    )
    os.system(command)

    

In [44]:
results = Parallel(n_jobs=60, prefer="threads")(delayed(setup)(i) for i in range(num_nodes))

In file included from stack-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
In file included from stack-benchmark.cpp:12:
project.hpp: In member function ‘bool Project::checkValidcall(Call)’:
project.hpp:104:3: warning: no return statement in function returning non-void [-Wreturn-type]
   }
   ^
mkdir: cannot create directory ‘workload’: File exists
mkdir: cannot create directory ‘4-7200-10’: File exists
mkdir: cannot create directory ‘stack’: File exists
mkdir: cannot create directory ‘workload’: File exists
mkdir: cannot create directory ‘4-7200-10’: File exists
mkdir: cannot create directory ‘stack’: File exists
CMake Deprecation Warning at /home/tejas/Optimistic-Replication/junction/CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument

In [45]:
result = subprocess.check_output('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)"', shell=True, text=True)

# Convert the output to a list by splitting on newlines
ip_list = result.splitlines()

In [46]:
ip_list

['10.128.0.29', '10.128.0.70', '10.128.0.54', '10.128.0.27']

In [47]:
def get_sorted_instance_ips():
    result = subprocess.check_output(
        "gcloud compute instances list --filter='name~^tmane-' "
        "--format='value(name,networkInterfaces[0].networkIP)'",
        shell=True
    )
    lines = result.decode().splitlines()
    # Parse into list of tuples: (name, ip)
    name_ip_pairs = [line.strip().split() for line in lines]
    # Sort by name: optrep-000, optrep-001, ...
    sorted_pairs = sorted(name_ip_pairs, key=lambda x: x[0])
    # Extract sorted IPs
    sorted_ips = [ip for name, ip in sorted_pairs]
    return sorted_ips

ip_list = get_sorted_instance_ips()
ip_string = " ".join(ip_list)

In [48]:
ip_string

'10.128.0.29 10.128.0.70 10.128.0.54 10.128.0.27'

In [49]:
def run_code(i):
    output_dir = f"../../wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/{usecase}"
    output_file = f"{output_dir}/result{i}.out"
    command = (
        f'gcloud compute ssh --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03} '
        f'--project ucr-ursa-major-lesani-lab '
        f'--command "cd Optimistic-Replication/tcp/build; '
        f'sudo bash -c \''
        f'mkdir -p {output_dir}; '
        f'nohup ./main-replication-protocols {i+1} {num_nodes} {num_txns} {write_percent} {usecase} {operationrate} {fialed_node_id} {ip_string} > {output_file} 2>&1 &'
        f'\'"'
    )
    print(command)
    output = subprocess.getoutput(command)
    print(output)



In [50]:
# results = Parallel(n_jobs=60)(delayed(run_code)(i) for i in range(num_nodes))

for i in range(num_nodes):
    run_code(i)


gcloud compute ssh --zone us-central1-a tejas@tmane-000 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-7200-10/stack; nohup ./main-replication-protocols 1 4 7200 10 stack 1000000000 3 10.128.0.29 10.128.0.70 10.128.0.54 10.128.0.27 > ../../wellcoordination/workload/4-7200-10/stack/result0.out 2>&1 &'"

gcloud compute ssh --zone us-central1-a tejas@tmane-001 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-7200-10/stack; nohup ./main-replication-protocols 2 4 7200 10 stack 1000000000 3 10.128.0.29 10.128.0.70 10.128.0.54 10.128.0.27 > ../../wellcoordination/workload/4-7200-10/stack/result1.out 2>&1 &'"

gcloud compute ssh --zone us-central1-a tejas@tmane-002 --project ucr-ursa-major-lesani-lab --command "cd Optimistic-Replication/tcp/build; sudo bash -c 'mkdir -p ../../wellcoordination/workload/4-7200-10

In [51]:
time.sleep(16.5)



kill_code(3)

time.sleep(13)


In [52]:
for i in range(num_nodes):
    kill_code(i)

main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found
main-replication-protocols: no process found


In [53]:

if not os.path.exists('../optrep_outputs'):
    os.mkdir('../optrep_outputs')

for i in [0,2, 3]:
    command = (
        f'gcloud compute scp --zone us-central1-a '
        f'{gcp_username}@tmane-{i:03}:/home/{gcp_username}/Optimistic-Replication/wellcoordination/workload/{num_nodes}-{num_txns}-{write_percent}/stack/result{i}.out '
        f'../optrep_outputs/{i}.txt '
        f'--project ucr-ursa-major-lesani-lab'
    )
    os.system(command)
    # break

In [54]:
# os.system('gcloud compute instances delete --zone=us-central1-a --quiet --project=ucr-ursa-major-lesani-lab $(gcloud compute instances list --project=ucr-ursa-major-lesani-lab --filter="name~^tmane-" --format="value(name)")')